In [2]:
import os

os.chdir('C:\\Users\\gowra\\like_lion\\2.신경망\\신경망(클래스)/code')

In [3]:
%pwd

'C:\\Users\\gowra\\like_lion\\2.신경망\\신경망(클래스)\\code'

#### cnn 클래스 만들기

    합성곱 신경망의 클래스는 AdamModel() 클래스의 파생 클래스로 만들어 보도록 하겠습니다. 
    AdamModel() 클래스는 MlpModel()클래스를 상속받기 때문에 
    이러한 상속 구조는 오히려 다양한 계층들을 쉽게 추가할 수 있는 구조를 제공한다는 점에서 큰 장점을 가지게 됩니다.


In [ ]:
%run adam_model.ipynb

In [ ]:
class CnnBasicModel(AdamModel):
    def init(self, name, dataset, hconfigs, show_maps=False):
        # 은닉계층에 대한 정보를 잘못 입력한 경우에 대해 처리하는 코드
        # 리스트가 맞는지, 첫번째 값이 리스트나 숫자가 맞는지 확인 > 아니라면 리스트로 감싸준다.
        if isinstance(hconfigs, list) and \
        not isinstance(hconfigs[0], (list, int)):
            hconfigs = [hconfigs]
        
        self.show_maps = show_maps
        self.need_maps = False
        self.kernels = []  # CNN에 있어 가중치
        
        super(CnnBasicModel, self).__init__(name, dataset, hconfigs)
        
        self.use_adam = True  # 아담 알고리즘 적용

    __init__ 객체 초기화 메서드를 정의하는 과정에서 중간 연산 과정의 결과 출력을 관장하는 show_maps 이 추가
    기본값은 False로 설정하여  필요한 경우에만 사용할 수 있도록 하였습니다.

    은닉 계층을 설정하는 과정에서 하나의 은닉 계층만 갖는 신경망 구조를 지정할 때 바깥 괄호 '[ ]' 를 빠뜨려 발생하는 오류를 막아주는 단계입니다. 
    앞서 우리가 프로그램에게 전달하는 값은 위와 같은 이중 구조입니다. 
    하지만 입력 단계에서 실수로 인해 ['conv', {'ksize':5}], ...  식으로 작성하게 되면 에러를 불러일으키게 됩니다. 
    그래서 isinstance() 함수를 통해 몇 가지를 확인하고 입력값을 정정하여 원활한 과정이 진행될 수 있도록 처리합니다.

    모델 초기화 과정 중에 생성되는 커널을 모아 저장할 객체 변수인 kernels를 공백 리스트로 초기화

    AdamModel() 클래스에서 False 였던 use_adam 플래그의 초깃값을 True로 바꿔 설정하여 자동적으로 아담 알고리즘이 적용되도록 설정
    아담 알고리즘을 설정하는 순서가 객체 초기화 과정 다음에 배치되어 있습니다. 
    객체 초기화 과정 이후에 다시 True로 설정이 되어야 효과를 발휘하기 때문입니다. 

#### 파라미터 생성, 순전파, 역전파에 대한 세 가지 메서드를 재정의 하는 과정

##### 파라미터 생성

In [ ]:
def cnn_basic_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)

    m_name = 'alloc_{}_layer'.format(layer_type)  
    method = getattr(self, m_name) # m_nmae을 가진 함수를 method에 저장
    pm, output_shape = method(input_shape, hconfig) # method에 저장된 함수에 input_shape, hconfig를 넣어 
                                                    # 결과를  pm, output_shape 로 받아온다.

    return pm, output_shape

CnnBasicModel.alloc_layer_param = cnn_basic_alloc_layer_param


    전체 신경망에 대한 파라미터 생성과 초기화를 담당하는 MlpModel() 클래스의 init_parameters() 메서드에 의해 호출되어 진다.

    하지만 기존의 과정은 다층 퍼셉트론의 완전 연결 방식이었기에 합성곱 계층과 풀링 계층이 추가가 될 수 있도록, 
    그리고 더 다양한 은닉 계층이 추가될 수 있도록 구조를 수정해야할 필요가 있다.


    메서드 이름에 의한 후킹 기법을 활용

    get_layer_type()라는 함수를 이용해 hconfig에 담긴 계층 이름 'conv'를 layer_type 값으로 얻어낸 후 
    format()를 활용하여 'alloc_conv_layer' 형식의 메서드 이름을 만들어 m_name에 저장 합니다. 

    그런 다음 getattr() 함수를 통해 m_name에 저장된 메서드를 꺼내어 method에 할당하고 
    현재 이 메서드의 반환값인 pm, output_shape을 얻어내어 주겠습니다.

    마지막으로 외부에서 정의된 메서드를 클래스 내부로 불러들여 줍니다.

    4종류인 'alloc_conv_layer()', 'alloc_max_layer()', 'alloc_full_layer()', 'alloc_avg_layer()' 메서드를 추가로 정의
    이 메서드는 공통적으로 input_shape, hconfig를 매개 변수로 전달받고, 
    필요한 처리를 마친 후에는 사전 구조체 형식의 파라미터 정보인 pm과 출력 형태 output_shape를 반환하게 됩니다.


##### 순전파 처리

In [ ]:
def cnn_basic_forward_layer(self, x, hconfig, pm):
    layer_type = get_layer_type(hconfig)
    
    m_name = 'forward_{}_layer'.format(layer_type)  
    method = getattr(self, m_name)  # method = forward_conv_layer()
    y, aux = method(x, hconfig, pm) # forward_conv_layer() 함수가 실행이 되어 결과 저장
    
    return y, aux

CnnBasicModel.forward_layer = cnn_basic_forward_layer


    매개 변수 hconfig를 get_layer_type()에 통과시켜 얻은 변수를 m_name에 할당하고, 
    이를 후킹하기 위해 getattr()를 활용하여 'forward_A_layer()' 메서드로 매개변수 x, hconfig, pm을 전달받아 y, aux를 반환한다. 

    순전파 처리 과정이기에 신경망 연산에 대한 결괏값과 역전파를 위한 보조지표 aux가 반환되었습니다.


##### 역전파 처리

In [ ]:
def cnn_basic_backprop_layer(self, G_y, hconfig, pm, aux):  
    layer_type = get_layer_type(hconfig)

    m_name = 'backprop_{}_layer'.format(layer_type)  
    method = getattr(self, m_name)
    G_input = method(G_y, hconfig, pm, aux)

    return G_input

CnnBasicModel.backprop_layer = cnn_basic_backprop_layer


    매개변수로 전달되어지는 값과 반환되는 값은 역전파 처리 수행 방식에 맞춰 필요한 값을 할당하여 줍니다. 
    하지만 지금 이 과정은 계층 하나에 대한 역전파 처리 과정입니다. 
    그렇기에 반환되어지는 값 또한 입력 텐서의 손실 기울기 G_input을 반환하여 주도록 합니다. 

    그림으로 살펴본 메서드 이름에 따른 후킹 기법의 가장 큰 틀을 대략적으로 구축하였습니다.  
    다음은 탐색된 메서드 'conv', 'full', 'avg', 'max'에 따른 파라미터 생성을 지원하도록 
    'alloc_A_layer' 메서드를 정의하도록 하겠습니다.


#### 완전 연결 계층의 파라미터 생성 과정 alloc_full_layer()

In [ ]:
def cnn_basic_alloc_full_layer(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = get_conf_param(hconfig, 'width', hconfig)

    weight = np.random.normal(0, self.rand_std, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

    return {'w': weight, 'b': bias}, [output_cnt]


CnnBasicModel.alloc_full_layer = cnn_basic_alloc_full_layer

    alloc_A_layer() 중에서 완전 연결 계층의 파라미터 생성 과정인 'alloc_full_layer()'를 구축하는 단계입니다.
    완전 연결 계층의 파라미터 생성을 구축하는 단계이니 만큼 결과적으로 input_cnt와 output_cnt를 확인해야 합니다. 
    그 과정에서 output_cnt의 경우에는 get_conf_param() 메서드에 의해 동작하게 된다.
    이 메서드는 cnn_basic_alloc_full_layer() 에서 전달 받은 은닉계층의 정보인 hconfig를 전달받아 output_cnt가 도출되어 집니다. 
    다음으로 가중치와 편향값을 생성하면 이 메서드는 완전 연결 계층의 파라미터 생성을 끝마치게 됩니다.


#### 합성곱 신경망의 파라미터 생성  alloc_conv_layer()

In [ ]:
def cnn_basic_alloc_conv_layer(self, input_shape, hconfig):
    assert len(input_shape) == 3
    xh, xw, xchn = input_shape
    kh, kw = get_conf_param_2d(hconfig, 'ksize')
    ychn = get_conf_param(hconfig, 'chn')

    kernel = np.random.normal(0, self.rand_std, [kh, kw, xchn, ychn])
    bias = np.zeros([ychn])

    if self.show_maps: self.kernels.append(kernel)

    return {'k': kernel, 'b': bias}, [xh, xw, ychn]


CnnBasicModel.alloc_conv_layer = cnn_basic_alloc_conv_layer

    전달 받게될 매개 변수는 input_shape와 hconfig입니다. 
    여기서는 assert라는 '가정설명문' 을 사용하게 되는데, 이는 뒤에 True가 아니면 AssertError를 반환시켜 줍니다. 

    그렇다면 왜 assert를 사용해야 하는 걸까요?
    우리는 간혹 어떠한 메서드에 특정 값 혹은 특정 형식만을 입력해야 하는 경우가 있습니다. 
    그래서 그 값이 아닌 다른 값이 들어오게 되는 경우라면 에러를  반환시켜 주어서 더 이상의 진행을 막아주어야 합니다. 
    사실 이러한 경우에는 if문 이라던지 예외처리를 통해 문제를 해결할 수도 있지만 이러한 가정 설명문 assert 또한 문제를 해결하는 또 하나의 방법이 될 수 있습니다. 

    그렇다면 왜 가정 설명문으로 input_shape의 길이가 3일때만 동작해야 할까요?
    그 이유는 보편적으로 최초의 이미지 채널값은 RGB 즉, 3이기 때문입니다. 
    이렇게 들어온 input_shape는 각 형식에 맞게 xh, xw, xchn 으로 각각 분할하여 할당시켜 주겠습니다. 

    다음은 매개변수로 들어온 hconfig와 입력값으로 전달받은 커널 사이즈에 맞게 kw, kh를 반환하여 줍니다. 
    이 과정에서 사용되  는 또 하나의 메서드인 get_conf_param_2d()는 잠시 후에 바로 정의하도록 하겠습니다. 
    지금은 단순히 현재 전달된 hconfig에 맞춰 kw,kh를 반환시  켜주는 메서드로 생각하면 됩니다.

    출력에 대한 채널값인 ychn를 get_conf_param()를 통해 얻어내어 줍니다. 
    이 메서드는 앞서 output_cnt를 생성한 메서드와 같은 기능을 가지고 있지만, 
    주어진 매개변수에 따라 다른 기능을 하도록 정의되어 있습니다. 

    그래서 매개변수로 전달된 'chn'이 할당되어 진다면 합성곱 신경망에 맞는 출력값을 반환하여 줍니다. 
    이제 매개변수로 전달된 input_shape와 hconfig를 통해 xh, xw, xchn, kh, kw, ychn를 구해주었습니다.

    이 값들을 바탕으로 합성곱 신경망의 파라미터인 커널과 편향을 생성하는 작업을 수행하여 주고, 
    초기 CnnBasicModel() 클래스에서 show_maps를 True로 설정한 경우에 커널값을 확인할 수 있도록 self.kernels를 통해 생성된 kernel을 저장하여 주겠습니다.
    그리고 반환값은 역시 파라미터인 커널과 편향을, 그리고 output_shape인 xh, xw, ychn을 묶어 반환시켜 주겠습니다.  
    그런데 output_shape은 출력에 대한 크기입니다. 하지만 지금 우리가 출력하고 있는 것은 입력에 대한 부분입니다.
    그 이유는, 지금 간단한 합성곱인 경우에는 same 패딩방식과 stride가 [1,1] 로서 xw = yw, xh = yh가 되기 때문입니다.


#### 풀링 레이어에 대한 파라미터 생성 과정 alloc_pool_layer()

In [ ]:
def cnn_basic_alloc_pool_layer(self, input_shape, hconfig):  
    assert len(input_shape) == 3
    xh, xw, xchn = input_shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    
    assert xh % sh == 0
    assert xw % sw == 0

    return {}, [xh//sh, xw//sw, xchn]

CnnBasicModel.alloc_max_layer = cnn_basic_alloc_pool_layer  
CnnBasicModel.alloc_avg_layer = cnn_basic_alloc_pool_layer



    풀링 과정이란 결과적으로 기존의 해상도를 축소시키는 과정이라 할 수 있습니다. 
    그리고 지금은 굉장히 간단한 합성곱 신경망을 다루는 과정이기에 건너뛰기 과정도 [1,1] 입니다. 

    그래서 가정 설명문 assert를 통해 조건을 걸어 주었습니다.
    즉 지금의 파라미터 생성과정과는 크게 연관이 없다는 것을 알 수 있습니다.
    그렇기에 반환되는 파라미터는 빈 사전 형식을 띄워주었습니다.

    또 여기서 구축한 풀링은 최대치 풀링과 평균치 풀링인 두 가지 방법이 있는데, 
    작업내용은 다르지 않으므로 하나의 함수를 두가지 메서드로 중복 등록 처리  하였습니다.

    이러한 방식은 메서드의 외부 등록 방법이 갖고 있는 또 하나의 장점이라 할 수 있습니다.


#### get_layer_type, get_conf_param, get_conf_param_2d 정의

##### get_layer_type

In [ ]:
def get_layer_type(hconfig):
    if not isinstance(hconfig, list): return 'full'
    return hconfig[0]

    전달되어진 매개변수 hconfig가 리스트 형식인지 확인하고, 리스트가 아닌 경우 'full'이라는 문자열을 반환하여 줍니다. 
    그럼 이 반환된 문자열은 layer_type으로 전달되어 '이름에 따른 후킹기법'에 의해 alloc_full_layer가 되며 이는 m_name 그리고 method로 동작하게 됩니다.

    만약 리스트인 경우에는 리스트의 가장 첫 번째 요소를 은닉 계층의 유형으로 보고합니다.
    
    입력하게 될 값을 떠올려 보면 [['full', {'width':30}], ['conv', {'ksize':5, 'chn':6}]] 과 같은 형태임을 알 수 있습니다. 
    그렇기에 리스트인 경우 가장 첫 번째 요소인 'full', 'conv', 'max' 등 과 같은 값을 전달하게 되어 format 기법을 통해 method를 정의할 수 있게 됩니다.


##### get_conf_param()

In [ ]:
def get_conf_param(hconfig, key, defval=None):
    if not isinstance(hconfig, list): return defval
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    return hconfig[1][key]

    output_cnt, ychn 를 반환하는데 사용되며 또 뒤에서 등장하는 func 를 반환하는데 사용되어 집니다. 
    이 함수는 결과적으로 전달되어진 은닉계층 정보에 대한 key에 저장된 value를 반환하는 과정입니다.

    예를들어 프로그램에 입력되어진 은닉 계층에 대한 정보가 ['full', {'width':30}] 라면 30을 반환하는 것 입니다.
    이 과정에서 여러가지 예외 상황들을 if문을 통해 검사하여 틀에 맞지 않는 값을 찾을 수 없을 때는 defval에 저장해둔 None값을 반환시켜 주는 과정을 추가하였습니다.


##### get_conf_param_2d()

In [ ]:
def get_conf_param_2d(hconfig, key, defval=None):
    if len(hconfig) <= 1: return defval
    if not key in hconfig[1]: return defval
    val = hconfig[1][key]
    if isinstance(val, list): return val
    return [val, val]

    스트라이드와 커널에 대한 너비와 높이값을 반환하는 함수를 정의하는 과정입니다. 
    이 메서드의 경우에는 앞서 정의한 메서드와 비슷하게 구조가 설계되어 있습니다. 
    전달되어지는 key 값에 따른 단일 value를 리스트 형식으로 반환하는 구조인데,
    예를 들어 전달되어지는 stride 혹은 ksize의 값이 1 인 경우 [1,1]의 형태로 반환시켜 줍니다.

    만약 val 값이 리스트 형식인 경우 리스트를 벗겨 반환하는 수식을 추가하여, 발생할 수 있는 에러를 사전에 차단하였습니다.


In [ ]:
def cnn_basic_forward_full_layer(self, x, hconfig, pm):
    if pm is None: return x, None
    
    x_org_shape = x.shape
    
    if len(x.shape) != 2:
        mb_size = x.shape[0]
        x = x.reshape([mb_size, -1])
        
    affine = np.matmul(x, pm['w']) + pm['b']
    y = self.activate(affine, hconfig)
    
    return y, [x, y, x_org_shape]

CnnBasicModel.forward_full_layer = cnn_basic_forward_full_layer

In [ ]:
def cnn_basic_backprop_full_layer(self, G_y, hconfig, pm, aux):
    if pm is None: return G_y

    x, y, x_org_shape = aux
    
    G_affine = self.activate_derv(G_y, y, hconfig)
    
    g_affine_weight = x.transpose()
    g_affine_input = pm['w'].transpose()
    
    G_weight = np.matmul(g_affine_weight, G_affine)
    G_bias = np.sum(G_affine, axis = 0)
    G_input = np.matmul(G_affine, g_affine_input)
    
    self.update_param(pm, 'w', G_weight)
    self.update_param(pm, 'b', G_bias)

    return G_input.reshape(x_org_shape)

CnnBasicModel.backprop_full_layer = cnn_basic_backprop_full_layer

In [ ]:
def cnn_basic_activate(self, affine, hconfig):
    if hconfig is None: return affine
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return affine
    elif func == 'relu':    return relu(affine)
    elif func == 'sigmoid': return sigmoid(affine)
    elif func == 'tanh':    return tanh(affine)
    else:                   assert 0
        
def cnn_basic_activate_derv(self, G_y, y, hconfig):
    if hconfig is None: return G_y
    
    func = get_conf_param(hconfig, 'actfunc', 'relu')
    
    if func == 'none':      return G_y
    elif func == 'relu':    return relu_derv(y) * G_y
    elif func == 'sigmoid': return sigmoid_derv(y) * G_y
    elif func == 'tanh':    return tanh_derv(y) * G_y
    else:                   assert 0

CnnBasicModel.activate = cnn_basic_activate
CnnBasicModel.activate_derv = cnn_basic_activate_derv

In [ ]:
def forward_conv_layer_adhoc(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    for i in range(kh):
                        for j in range(kw):
                            rx = r + i - (kh-1) // 2
                            cx = c + j - (kw-1) // 2
                            if rx < 0 or rx >= xh: continue
                            if cx < 0 or cx >= xw: continue
                            for xm in range(xchn):
                                kval = pm['k'][i][j][xm][ym]
                                ival = x[n][rx][cx][xm]
                                conv[n][r][c][ym] += kval * ival

    y = self.activate(conv + pm['b'], hconfig)
    
    return y, [x, y]

In [ ]:
def forward_conv_layer_better(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape
    
    conv = np.zeros((mb_size, xh, xw, ychn))

    bh, bw = (kh-1)//2, (kw-1)//2
    eh, ew = xh + kh - 1, xw + kw - 1
    
    x_ext = np.zeros((mb_size, eh, ew, xchn))
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x
    
    k_flat = pm['k'].transpose([3, 0, 1, 2]).reshape([ychn, -1])
    
    for n in range(mb_size):
        for r in range(xh):
            for c in range(xw):
                for ym in range(ychn):
                    xe_flat = x_ext[n, r:r + kh, c:c + kw, :].flatten()
                    conv[n, r, c, ym] = (xe_flat*k_flat[ym]).sum()
                    
    y = self.activate(conv + pm['b'], hconfig)
    
    return y, [x, y]

In [ ]:
def cnn_basic_forward_conv_layer(self, x, hconfig, pm):
    mb_size, xh, xw, xchn = x.shape
    kh, kw, _, ychn = pm['k'].shape  # 커널은 높이, 너비, 입력 채널값, 출력 채널값을 갖는다.
    
    x_flat = get_ext_regions_for_conv(x, kh, kw)
    k_flat = pm['k'].reshape([kh*kw*xchn, ychn])
    conv_flat = np.matmul(x_flat, k_flat)
    conv = conv_flat.reshape([mb_size, xh, xw, ychn])

    y = self.activate(conv + pm['b'], hconfig)

    if self.need_maps: self.maps.append(y)
    
    return y, [x_flat, k_flat, x, y]

CnnBasicModel.forward_conv_layer = cnn_basic_forward_conv_layer

In [ ]:
def cnn_basic_backprop_conv_layer(self, G_y, hconfig, pm, aux):
    x_flat, k_flat, x, y = aux
    
    kh, kw, xchn, ychn = pm['k'].shape
    mb_size, xh, xw, _ = G_y.shape
    
    G_conv = self.activate_derv(G_y, y, hconfig)

    G_conv_flat = G_conv.reshape(mb_size*xh*xw, ychn)

    g_conv_k_flat = x_flat.transpose()
    g_conv_x_flat = k_flat.transpose()
    
    G_k_flat = np.matmul(g_conv_k_flat, G_conv_flat)
    G_x_flat = np.matmul(G_conv_flat, g_conv_x_flat)
    G_bias = np.sum(G_conv_flat, axis = 0)
    
    G_kernel = G_k_flat.reshape([kh, kw, xchn, ychn])
    G_input = undo_ext_regions_for_conv(G_x_flat, x, kh, kw)
    
    self.update_param(pm, 'k', G_kernel)
    self.update_param(pm, 'b', G_bias)
    
    return G_input

CnnBasicModel.backprop_conv_layer = cnn_basic_backprop_conv_layer

In [ ]:
def get_ext_regions_for_conv(x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = get_ext_regions(x, kh, kw, 0)
    regs = regs.transpose([2, 0, 1, 3, 4, 5])
    
    return regs.reshape([mb_size*xh*xw, kh*kw*xchn])

def get_ext_regions(x, kh, kw, fill):
    mb_size, xh, xw, xchn = x.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    x_ext = np.zeros((mb_size, eh, ew, xchn), dtype = 'float32') + fill
    x_ext[:, bh:bh + xh, bw:bw + xw, :] = x
    
    regs = np.zeros((xh, xw, mb_size*kh*kw*xchn), dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            regs[r, c, :] = x_ext[:, r:r + kh, c:c + kw, :].flatten()

    return regs.reshape([xh, xw, mb_size, kh, kw, xchn])

In [ ]:
def undo_ext_regions_for_conv(regs, x, kh, kw):
    mb_size, xh, xw, xchn = x.shape

    regs = regs.reshape([mb_size, xh, xw, kh, kw, xchn])
    regs = regs.transpose([1, 2, 0, 3, 4, 5])
    
    return undo_ext_regions(regs, kh, kw)

def undo_ext_regions(regs, kh, kw):
    xh, xw, mb_size, kh, kw, xchn = regs.shape
    
    eh, ew = xh + kh - 1, xw + kw - 1
    bh, bw = (kh-1)//2, (kw-1)//2

    gx_ext = np.zeros([mb_size, eh, ew, xchn], dtype = 'float32')

    for r in range(xh):
        for c in range(xw):
            gx_ext[:, r:r + kh, c:c + kw, :] += regs[r, c]

    return gx_ext[:, bh:bh + xh, bw:bw + xw, :]

In [ ]:
def cnn_basic_forward_avg_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw

    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4)
    x3 = x2.reshape([-1, sh*sw])
    
    y_flat = np.average(x3, 1)
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y)

    return y, None

def cnn_basic_backprop_avg_layer(self, G_y, hconfig, pm, aux):
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten() / (sh * sw)

    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32')
    for i in range(sh*sw):
        gx1[:, i] = gy_flat
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])

    G_input = gx3.reshape([mb_size, xh, xw, chn])
        
    return G_input

CnnBasicModel.forward_avg_layer = cnn_basic_forward_avg_layer
CnnBasicModel.backprop_avg_layer = cnn_basic_backprop_avg_layer

In [ ]:
def cnn_basic_forward_max_layer(self, x, hconfig, pm):
    mb_size, xh, xw, chn = x.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    yh, yw = xh // sh, xw // sw

    x1 = x.reshape([mb_size, yh, sh, yw, sw, chn])
    x2 = x1.transpose(0, 1, 3, 5, 2, 4)
    x3 = x2.reshape([-1, sh*sw])

    idxs = np.argmax(x3, axis = 1)
    y_flat = x3[np.arange(mb_size*yh*yw*chn), idxs]
    y = y_flat.reshape([mb_size, yh, yw, chn])
    
    if self.need_maps: self.maps.append(y)

    return y, idxs

def cnn_basic_backprop_max_layer(self, G_y, hconfig, pm, aux):
    idxs = aux
    
    mb_size, yh, yw, chn = G_y.shape
    sh, sw = get_conf_param_2d(hconfig, 'stride')
    xh, xw = yh * sh, yw * sw
    
    gy_flat = G_y.flatten()

    gx1 = np.zeros([mb_size*yh*yw*chn, sh*sw], dtype = 'float32')
    gx1[np.arange(mb_size*yh*yw*chn), idxs] = gy_flat[:]
    gx2 = gx1.reshape([mb_size, yh, yw, chn, sh, sw])
    gx3 = gx2.transpose([0, 1, 4, 2, 5, 3])

    G_input = gx3.reshape([mb_size, xh, xw, chn])
        
    return G_input

CnnBasicModel.forward_max_layer = cnn_basic_forward_max_layer
CnnBasicModel.backprop_max_layer = cnn_basic_backprop_max_layer

In [ ]:
def cnn_basic_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    
    self.need_maps = self.show_maps
    self.maps = []

    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)

    if self.show_maps:
        for kernel in self.kernels:
            kh, kw, xchn, ychn = kernel.shape
            grids = kernel.reshape([kh, kw, -1]).transpose(2, 0, 1)
            draw_images_horz(grids[0:5, :, :])

        for pmap in self.maps:
            draw_images_horz(pmap[:, :, :, 0])
        
    self.dataset.visualize(deX, est, deY)

    self.need_maps = False
    self.maps = None

CnnBasicModel.visualize = cnn_basic_visualize